## OpenEO Basics

In this notebook, we will introduce you to the most basic functionality of openEO. Please do refer to our [starters documentation](https://docs.openeo.cloud/getting-started/python/#installation) for a more elaborate introduction, or for our [detailed documentation](https://open-eo.github.io/openeo-python-client/index.html) on github.

We will first install and import the openeo package. Then we can make a connection to a backend. Note that you can either choose a [specific backend](https://hub.openeo.org/) you want to run on, or you can let the aggregator choose one for you depending on the functionality that you are using. We are going to choose the aggregator for now, which can be accessed through the URL *openeo.cloud*.

In [ ]:
# !pip install openeo

In [1]:
import openeo

print(openeo.client_version())
base_path = Path("results/basics")

0.9.1


In [2]:
connection = openeo.connect("openeo.cloud").authenticate_oidc()

Authenticated using refresh token.


We can now start loading a collection. An overview of collections can be found here: https://docs.openeo.cloud/data-collections/

The method load_collection is documented here: https://docs.openeo.cloud/processes/#load_collection

In [3]:
s2_cube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west":5.1,"east":5.2,"south":51.1,"north":51.2},
    temporal_extent=["2020-01-01","2020-12-31"],
    bands=["B02","B04","B08"]
)

Generally, the first step after loading in a collection will be cloud masking. There are many different ways to perform cloud masking, depending on the collection used and the criteria of your specific use case. There is some documentation on cloud masking [here](https://open-eo.github.io/openeo-python-client/cookbook/ard.html), as well as a notebook that exploits a cloud masking technique based on the SCL layer of Sentinel-2, which can be found [here](https://github.com/VITObelgium/notebook-samples/blob/master/Terrascope/Advanced/OpenEODataCubesMasking.ipynb). At Vito, we have developed our own cloud masking technique based on sen2cor, which is applied [here](https://github.com/openEOPlatform/SRR2_notebooks/blob/main/UC3%20-%20Crop%20type%20feature%20engineering%20using%20random%20forest.ipynb)

There are two ways to download results in openEO. One is using a synchronous job, and one is using a batch job. Synchronous jobs are quick, but can only be used for very light jobs. Heavy jobs require the use of batch processing.

We will first demonstrate downloading results using a synchronous job

Note: we used max_time() so that you only need to download 1 time interval just to speed up the download process for this particular demo. To see what the method does and to see more methods of a DataCube, you can check out:
https://open-eo.github.io/openeo-python-client/api.html#openeo.rest.datacube.DataCube.max_time

In [ ]:
s2_cube.max_time().download(str(base_path / "s2_cube_syn.tif")),format="Gtiff")

Downloading results using a batch job works as follows, and can be done in two different ways:

In [ ]:
job = s2_cube.ndvi(nir="B08", red="B04").execute_batch(format="Gtiff")

In [8]:
job = s2_cube.ndvi(nir="B08", red="B04").send_job(format="Gtiff")
job.start_job()
job.describe_job()
# job.get_results().download_file(str(base_path / "s2_cube_batch.tif"))

{'created': '2022-01-13T09:19:50Z',
 'id': '84a4c17c-8c29-475f-a6a7-0edbb7b335d2',
 'process': {'process_graph': {'loadcollection1': {'arguments': {'bands': ['B02',
      'B04',
      'B08'],
     'id': 'TERRASCOPE_S2_TOC_V2',
     'spatial_extent': {'east': 5.2,
      'north': 51.2,
      'south': 51.1,
      'west': 5.1},
     'temporal_extent': ['2020-01-01', '2020-12-31']},
    'process_id': 'load_collection'},
   'ndvi1': {'arguments': {'data': {'from_node': 'loadcollection1'},
     'nir': 'B08',
     'red': 'B04'},
    'process_id': 'ndvi',
    'result': True}}},
 'status': 'queued',
 'updated': '2022-01-13T09:22:00Z'}

When our job is running we can retrieve the status, a description, or we can stop a job.

In [24]:
print(job.status())

running


In [11]:
job.stop_job()

However, generally we are interested in more than just loading and downloading data. One of the things we can do is to filter a datacube, temporally, spatially or on bands.

In [5]:
b2_filtered_cropped = s2_cube.filter_temporal(["2020-02-01","2020-02-28"]).band("B02").filter_bbox(west=5.15,east=5.16,south=51.14,north=51.16, crs=4326)

When filtering on bands, we can also do band math.

In [26]:
B04 = s2_cube.band("B04")
B08 = s2_cube.band("B08")
ndvi_cube = (B08 - B04) / (B08 + B04)
ndvi_cube.max_time().download(str(base_path / "ndvi.tif"),format="GTiff")

Applying mathematical operations

In [ ]:
s2_absolute = s2_cube.apply("absolute")
s2_lambda = s2_cube.apply(lambda x: x.absolute().cos())

In [45]:
s2_red = s2_cube.reduce_dimension(dimension="t",reducer="max")

In [31]:
from openeo.processes import array_element
def callback(data):
    band1 = array_element(data, index=0)
    band2 = array_element(data, index=1)
    return band1 + 1.2 * band2

s2_cube.reduce_dimension(dimension="bands",reducer=callback)

In [ ]:
def callback2(data):
    return data.mean()

s2_cube.reduce_dimension(dimension="t",reducer=callback2)

Sampling features. For more information, see [here](https://open-eo.github.io/openeo-python-client/cookbook/sampling.html)

In [ ]:
with open(Path("resources/soy_2019.geojson")) as fn:
    pol = fn.readlines()[0]

sampled_features = ndvi_cube.filter_spatial(json.loads(pol))
job = sampled_features.send_job(
    title="Extract points from SHub",
    out_format="netCDF",
    sample_by_feature=True)
job.start_job()

In [52]:
job.status()

'finished'